In [1]:
import googleapiclient.discovery
import googleapiclient.errors
import numpy as np
import pandas as pd
import glob
import nltk 
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from langdetect import detect
import shutil
import random
from wordcloud import WordCloud
import gensim
import seaborn as sns
from textblob import TextBlob
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.datasets import load_files
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Archisa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Archisa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

In [3]:
train_df = train_df.head(10000)

In [4]:
train_df.shape

(10000, 2)

In [5]:
def remove_punctuation(text):
    # Define the pattern to match punctuation
    punctuation_pattern = r'[^\w\s]'
    # Replace punctuation with an empty string
    text_without_punctuation = re.sub(punctuation_pattern, '', text)
    # Normalize whitespace
    normalized_text = re.sub(r'\s+', ' ', text_without_punctuation)
    return normalized_text

# Text Processing
def preprocess_text(text, method='snowballstemming'):
    # Tokenize and lowercase
    tokens = word_tokenize(text.lower())

    # Define the set of stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords and punctuation
    tokens = [remove_punctuation(token) for token in tokens if token.isalnum() and token not in stop_words]
    
    if method == 'snowballstemming':
        stemmer = nltk.stem.SnowballStemmer('english')
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'porterstemming':
        stemmer = nltk.stem.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'lemmatization':
        lemmatizer = nltk.stem.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [6]:
train_df['ssprocessing_text'] = train_df['Review'].apply(preprocess_text, method='snowballstemming')
train_df['psprocessing_text'] = train_df['Review'].apply(preprocess_text, method='porterstemming')
train_df['lemprocessing_text'] = train_df['Review'].apply(preprocess_text, method='lemmatization')

In [7]:
train_df

,overall,Review,ssprocessing_text,psprocessing_text,lemprocessing_text
0,5,I love these glitter pens. They sparkle deligh...,love glitter pen sparkl delight page brilliant...,love glitter pen sparkl delight page brilliant...,love glitter pen sparkle delightfully page bri...
1,5,It works well with my machine. I use mostly c...,work well machin use most cone,work well machin use mostli cone,work well machine use mostly cone
2,5,"This is a great assortment of colors, though t...",great assort color though lot pink mix still c...,great assort color though lot pink mix still c...,great assortment color though lot pink mix sti...
3,5,Just what I was looking for.,look,look,looking
4,5,I make 400 birds for the hospital each month.,make 400 bird hospit month,make 400 bird hospit month,make 400 bird hospital month
...,...,...,...,...,...
9995,5,"Lovely yarn, very fast delivery, I am so pleas...",love yarn fast deliveri pleas buy,love yarn fast deliveri pleas buy,lovely yarn fast delivery pleased buy
9996,5,These perfectly match some charms I ordered to...,perfect match charm order make photo pendant like,perfectli match charm order make photo pendant...,perfectly match charm ordered make photo penda...
9997,5,Perfect for my project.,perfect project,perfect project,perfect project
9998,5,Great product!,great product,great product,great product


In [8]:
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense
from sklearn.model_selection import train_test_split

import gensim

In [9]:
tokenized_df = train_df.copy()

# Iterate over each column
for column in train_df.columns:
    # Check if the column is not 'overall'
    if column != 'overall':
        # Tokenize values in the column
        tokenized_values = train_df[column].apply(lambda x: word_tokenize(x))
        # Add tokenized values to the new DataFrame
        tokenized_df[column] = tokenized_values

tokenized_df

# word2vec_model = gensim.models.Word2Vec(subset_df["tokens"].tolist(), min_count=5, window=9, vector_size=100)

,overall,Review,ssprocessing_text,psprocessing_text,lemprocessing_text
0,5,"[I, love, these, glitter, pens, ., They, spark...","[love, glitter, pen, sparkl, delight, page, br...","[love, glitter, pen, sparkl, delight, page, br...","[love, glitter, pen, sparkle, delightfully, pa..."
1,5,"[It, works, well, with, my, machine, ., I, use...","[work, well, machin, use, most, cone]","[work, well, machin, use, mostli, cone]","[work, well, machine, use, mostly, cone]"
2,5,"[This, is, a, great, assortment, of, colors, ,...","[great, assort, color, though, lot, pink, mix,...","[great, assort, color, though, lot, pink, mix,...","[great, assortment, color, though, lot, pink, ..."
3,5,"[Just, what, I, was, looking, for, .]",[look],[look],[looking]
4,5,"[I, make, 400, birds, for, the, hospital, each...","[make, 400, bird, hospit, month]","[make, 400, bird, hospit, month]","[make, 400, bird, hospital, month]"
...,...,...,...,...,...
9995,5,"[Lovely, yarn, ,, very, fast, delivery, ,, I, ...","[love, yarn, fast, deliveri, pleas, buy]","[love, yarn, fast, deliveri, pleas, buy]","[lovely, yarn, fast, delivery, pleased, buy]"
9996,5,"[These, perfectly, match, some, charms, I, ord...","[perfect, match, charm, order, make, photo, pe...","[perfectli, match, charm, order, make, photo, ...","[perfectly, match, charm, ordered, make, photo..."
9997,5,"[Perfect, for, my, project, .]","[perfect, project]","[perfect, project]","[perfect, project]"
9998,5,"[Great, product, !]","[great, product]","[great, product]","[great, product]"


In [10]:
train_df_ss = tokenized_df[['overall', 'ssprocessing_text']].copy()
train_df_ss

,overall,ssprocessing_text
0,5,"[love, glitter, pen, sparkl, delight, page, br..."
1,5,"[work, well, machin, use, most, cone]"
2,5,"[great, assort, color, though, lot, pink, mix,..."
3,5,[look]
4,5,"[make, 400, bird, hospit, month]"
...,...,...
9995,5,"[love, yarn, fast, deliveri, pleas, buy]"
9996,5,"[perfect, match, charm, order, make, photo, pe..."
9997,5,"[perfect, project]"
9998,5,"[great, product]"


In [11]:
train_df_ps = tokenized_df[['overall', 'psprocessing_text']].copy()
train_df_ps

,overall,psprocessing_text
0,5,"[love, glitter, pen, sparkl, delight, page, br..."
1,5,"[work, well, machin, use, mostli, cone]"
2,5,"[great, assort, color, though, lot, pink, mix,..."
3,5,[look]
4,5,"[make, 400, bird, hospit, month]"
...,...,...
9995,5,"[love, yarn, fast, deliveri, pleas, buy]"
9996,5,"[perfectli, match, charm, order, make, photo, ..."
9997,5,"[perfect, project]"
9998,5,"[great, product]"


In [12]:
train_df_lem = tokenized_df[['overall', 'lemprocessing_text']].copy()
train_df_lem

,overall,lemprocessing_text
0,5,"[love, glitter, pen, sparkle, delightfully, pa..."
1,5,"[work, well, machine, use, mostly, cone]"
2,5,"[great, assortment, color, though, lot, pink, ..."
3,5,[looking]
4,5,"[make, 400, bird, hospital, month]"
...,...,...
9995,5,"[lovely, yarn, fast, delivery, pleased, buy]"
9996,5,"[perfectly, match, charm, ordered, make, photo..."
9997,5,"[perfect, project]"
9998,5,"[great, product]"
